In [1]:
'''
- ImageDataGenerator로 간단하게 이미지 분류하기
* train/validation/test로 나누어서 수행할 것
* train/validation/test의 폴더 이름(클래스 이름)이 전부 같아야 함

'''

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
np.random.seed(7)

# train 데이터셋
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'C:/Users/Lecture/MNIST/trainingSet',
        target_size=(28, 28),
        batch_size=5,
        class_mode='categorical')

# valid 데이터셋
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
        'C:/Users/Lecture/MNIST/validSet',  # 경로
        target_size=(28, 28),    # 이미지 사이즈
        batch_size=5,  # 한 에폭에 사용될 배치의 크기
        class_mode='categorical')# 다중 클래스이므로 categorical


# 모델 설계
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit_generator(
        train_generator,
        steps_per_epoch=60,  # 총 1500개의 훈련 샘플, 배치사이즈 25 = 1500/25 = 60
        epochs=10,
        validation_data=valid_generator,
        validation_steps=12) # 총 60개의 테스트 샘플, 배치사이즈 5 = 60/5 = 12

# train/validation 학습 평가
scores = model.evaluate_generator(valid_generator, steps=12)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# test 데이터 평가
# test 데이터 생성
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'C:/Users/Lecture/MNIST/testSet',
        target_size=(28, 28),   
        shuffle=False) 

# 예측
pred = model.predict_generator(test_generator)
predict = np.zeros((len(pred), len(test_generator.class_indices.keys())))
for i in range(len(pred)):
    tmp = np.where(pred[i] == pred[i].max())[0][0]
    predict[i][tmp] = 1
        
test_filenames = test_generator.filenames

for i in range(len(test_filenames)):
    test_filenames[i] = test_filenames[i].partition('\\')[0]

test_classnames = test_generator.class_indices
test_classnames = list(test_classnames.keys())
test_classnames.sort()

# 열의 이름은 test 파일을 어디로 분류했는지 표현
# 각 행의 이름은 test 파일의 category를 표현
submission = pd.DataFrame(predict, index = test_filenames, columns = test_classnames)
submission.to_csv("test.csv", encoding='euc-kr')

colnames = submission.columns

right=0
for i in range(len(colnames)):
    check = submission.loc[colnames[i]][colnames[i]]
    temp = check.isin([1]).tolist()
    cnt = temp.count(True)
    right += cnt
    
accuracy = right/len(predict)
print("test accuracy: " + str(accuracy))

Using TensorFlow backend.


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:/Users/Lecture/MNIST/trainingSet'